In [9]:
# Colabで必要なパッケージのインストール
%pip install datasets pandas tqdm transformers vllm --quiet

# 必要なライブラリのインポート
import os
import datasets
import pandas as pd
import json
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from vllm import LLM, SamplingParams
from itertools import islice

# 分散環境の設定
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29500"
os.environ["GLOO_SOCKET_IFNAME"] = "lo"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
run_index = 1 # 複数回実行する予定ですので、結果を保存するためにrun_indexを使用します
# モデルとパラメータの設定
inference_model = "Qwen/Qwen3-8B"  # TODO: 使用したい推論モデルに変更してください
inference_temperature = 0.3  # 温度パラメータ（低いほど決定的な出力になります）
inference_max_tokens = 1024 # 生成する最大トークン数
inference_batch_size = 4
inference_tp, inference_pp, inference_dp = 1, 1, 1  # テンソル並列、パイプライン並列、データ並列の設定
save_per_batch = 1  # `save_per_batch`バッチごとに結果を保存

judgement_model = "Qwen/Qwen3-8B"  # TODO: 使用したい判定モデルに変更してください
judgement_temperature = 0.1  # 温度パラメータ（低いほど決定的な出力になります）
judgement_max_tokens = 50
judgement_batch_size = 4
judgement_tp, judgement_pp, judgement_dp = 1, 1, 1

# データセットサイズの設定（固定値）
cot_dataset_size = 3.3e9
genselect_dataset_size = 5.66e5

# 処理範囲の設定
start_from_percentage = 0  # 0.5 = 50%から開始
end_at_percentage = 1.0  # 1.0 = 100%まで処理

# 出力ディレクトリの設定
output_dir = "./results"
inference_dir = f"{output_dir}/inference/run_{run_index}"  # 推論結果の一時保存用
judgement_dir = f"{output_dir}/judgement/run_{run_index}"  # 判定結果の一時保存用
os.makedirs(inference_dir, exist_ok=True)
os.makedirs(judgement_dir, exist_ok=True)


In [11]:
# プロンプトの定義
# 推論LLM用のプロンプト（CoT形式）
inference_cot_prompt = (
    "You are a highly skilled mathematician known for clear and rigorous reasoning.\n"
    "Given the following math question, provide a step-by-step analysis of your thought process, followed by the final answer.\n"
    "Question:\n"
    "{question}\n"
    "Please respond with only your reasoning steps and the final answer. Do not include any extraneous text or explanations outside your solution."
)

inference_genselect_prompt = (
    "You are a highly skilled mathematician known for clear and rigorous reasoning.\n"
    "You are given a math question along with several candidate answers.\n"
    "Analyze each candidate solution, explain your reasoning, and then state which candidate is correct as your final answer.\n"
    "Question and candidate solutions:\n"
    "{question}\n"
    "Please respond with only your analysis and the final answer. The final answer must be one of the provided candidate solutions. Do not include any extraneous text."
)

judgement_cot_prompt = (
    "You are a mathematics expert tasked with evaluating a user's solution.\n"
    "You will be given a question, the correct answer, and the user's solution (including their reasoning and final answer).\n"
    "Determine if BOTH the reasoning and the final answer in the user's solution are correct.\n"
    "Question:\n"
    "{question}\n"
    "Correct answer:\n"
    "{correct_answer}\n"
    "User's solution:\n"
    "{solution}\n"
    "Reply with only 'yes' if both are correct, or 'no' if either is incorrect. Do not include any other text."
)

judgement_genselect_prompt = (
    "You are a mathematics expert tasked with evaluating a user's solution.\n"
    "You will be given a question with candidate solutions, the correct answer, and the user's analysis and final answer.\n"
    "Determine if BOTH the reasoning and the final answer in the user's solution are correct.\n"
    "Question and candidate solutions:\n"
    "{question}\n"
    "Correct answer:\n"
    "{correct_answer}\n"
    "User's solution:\n"
    "{solution}\n"
    "Reply with only 'yes' if both are correct, or 'no' if either is incorrect. Do not include any other text."
)


In [12]:
# vLLMを使用した推論と判定の関数定義

def vllm_inference(llm, prompts, temperature=0.3, max_tokens=1024):
    """
    推論用の関数：与えられたプロンプトに対して推論を実行
    """
    sampling_params = SamplingParams(
        temperature=temperature,
        max_tokens=max_tokens,
        n=1,  # 各プロンプトに対する生成数
    )
    results = []
    outputs = llm.generate(prompts, sampling_params)
    for output in outputs:
        results.append(output.outputs[0].text)
    return results

def vllm_judgement(llm, prompts, temperature=0.1, max_tokens=1024):
    """
    判定用の関数：与えられたプロンプトに対して判定を実行
    """
    sampling_params = SamplingParams(
        temperature=temperature,
        max_tokens=max_tokens,
        n=1,
    )
    results = []
    outputs = llm.generate(prompts, sampling_params)
    for output in outputs:
        results.append(output.outputs[0].text)
    return results


In [13]:
# データセット全体に対する推論処理の関数
def inference(inf_dataset, inference_batch_size, save_per_batch, inference_temperature, inference_max_tokens, inference_prompt, inference_dir, dataset_size):
    if not os.path.exists(inference_dir):
        os.makedirs(inference_dir)

    inference_collection = []
    start_from_batch_index = int(dataset_size * start_from_percentage // inference_batch_size)
    end_at_batch_index = int(dataset_size * end_at_percentage // inference_batch_size)

    # filter the inf_dataset by the problem_type column to be has_answer_extracted
    inf_dataset = inf_dataset.filter(lambda x: x['problem_type'] == 'has_answer_extracted')
    i = 1
    for data_batch in tqdm(inf_dataset.iter(batch_size=inference_batch_size), desc="Inferencing"):
        if i < start_from_batch_index:
            i += 1
            continue
        if i >= end_at_batch_index:
            break
        
        if i % (save_per_batch) == 0 and os.path.exists(f"{inference_dir}/inference_{i}.json"):
            i += 1
            continue

        inference_prompts = [inference_prompt.format(question=question) for question in data_batch["problem"]]
        inference_results = vllm_inference(llm, inference_prompts, inference_temperature, inference_max_tokens)
        # add the inference results to data_batch, make a new column called 'inference'
        data_batch['inference'] = inference_results
        # data_batch is a dictionary, convert it to a list of dictionaries
        data_batch = [{k: v[j] for k, v in data_batch.items()} for j in range(inference_batch_size)]
        inference_collection.extend(data_batch)

        # save the temporary inference results
        if i % (save_per_batch) == 0:
            # save as pandas dataframe
            with open(f"{inference_dir}/inference_{i}.json", "w") as f:
                json.dump(inference_collection, f)
            inference_collection.clear()
        i += 1


In [ ]:
# 判定処理の関数
def judgement(jud_model, judgement_batch_size, judgement_temperature, judgement_max_tokens, judgement_prompt, inference_dir, judgement_dir):
    if not os.path.exists(judgement_dir):
        os.makedirs(judgement_dir)

    for inf_filename in tqdm(os.listdir(inference_dir)):
        judgement_filename = inf_filename.replace("inference", "judgement")
        if os.path.exists(f"{judgement_dir}/{judgement_filename}"):
            continue

        judgement_collection = []
        with open(f"{inference_dir}/{inf_filename}", "r") as f:
            inf_results = json.load(f)
        num_rows = len(inf_results)
        for index in tqdm(range(0, num_rows, judgement_batch_size), desc="Judging"):
            batch = inf_results[index:index+judgement_batch_size]
            question = [item['problem'] for item in batch]
            correct_answer = [item['generated_solution'] for item in batch]
            solution = [item['inference'] for item in batch]
            judgement_prompts = [judgement_prompt.format(question=q, correct_answer=ca, solution=s) for q, ca, s in zip(question, correct_answer, solution)]
            judgement_results = vllm_judgement(jud_model, judgement_prompts, judgement_temperature, judgement_max_tokens)
            for i, item in enumerate(batch):
                item['judgement'] = judgement_results[i]
            judgement_collection.extend(batch)
        with open(f"{judgement_dir}/{judgement_filename}", "w") as f:
            json.dump(judgement_collection, f)


In [ ]:
# メイン処理の実行

# 推論モデルのロード
print("推論モデルをロード中...")
llm = LLM(
    model=inference_model,
    tensor_parallel_size=inference_tp,
    pipeline_parallel_size=inference_pp,
    gpu_memory_utilization=0.95
)

# CoTデータセットの処理
print("CoTデータセットの処理を開始...")
cot_dataset = datasets.load_dataset("nvidia/OpenMathReasoning", split='cot', streaming=True)
inference(
    cot_dataset,
    inference_batch_size,
    save_per_batch,
    inference_temperature,
    inference_max_tokens,
    inference_cot_prompt,
    inference_dir + "/cot",
    cot_dataset_size
)
del cot_dataset  # メモリの解放

# GenSelectデータセットの処理
print("GenSelectデータセットの処理を開始...")
genselect_dataset = datasets.load_dataset("nvidia/OpenMathReasoning", split='genselect', streaming=True)
inference(
    genselect_dataset,
    inference_batch_size,
    save_per_batch,
    inference_temperature,
    inference_max_tokens,
    inference_genselect_prompt,
    inference_dir + "/genselect",
    genselect_dataset_size
)
del genselect_dataset  # メモリの解放
del llm
torch.cuda.empty_cache() # flush the caching allocator


In [15]:
# 判定モデルのロード
print("判定モデルをロード中...")
llm = LLM(
    model=judgement_model,
    tensor_parallel_size=judgement_tp,
    pipeline_parallel_size=judgement_pp,
    gpu_memory_utilization=0.95
)

# CoT結果の判定
print("CoT結果の判定を開始...")
judgement(
    llm,
    judgement_batch_size,
    judgement_temperature,
    judgement_max_tokens,
    judgement_cot_prompt,
    inference_dir + "/cot",
    judgement_dir + "/cot"
)

# GenSelect結果の判定
print("GenSelect結果の判定を開始...")
judgement(
    llm,
    judgement_batch_size,
    judgement_temperature,
    judgement_max_tokens,
    judgement_genselect_prompt,
    inference_dir + "/genselect",
    judgement_dir + "/genselect"
)

print("全ての処理が完了しました！")


判定モデルをロード中...
INFO 07-21 12:04:44 [config.py:841] This model supports multiple tasks: {'reward', 'generate', 'classify', 'embed'}. Defaulting to 'generate'.
INFO 07-21 12:04:44 [config.py:1472] Using max model len 40960
INFO 07-21 12:04:44 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 07-21 12:04:45 [core.py:526] Waiting for init message from front-end.
INFO 07-21 12:04:45 [core.py:69] Initializing a V1 LLM engine (v0.9.2) with config: model='Qwen/Qwen3-8B', speculative_config=None, tokenizer='Qwen/Qwen3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=40960, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


INFO 07-21 12:04:46 [parallel_state.py:1076] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
WARNING 07-21 12:04:46 [topk_topp_sampler.py:59] FlashInfer is not available. Falling back to the PyTorch-native implementation of top-p & top-k sampling. For the best performance, please install FlashInfer.
INFO 07-21 12:04:46 [gpu_model_runner.py:1770] Starting to load model Qwen/Qwen3-8B...
INFO 07-21 12:04:46 [gpu_model_runner.py:1775] Loading model from scratch...
INFO 07-21 12:04:46 [cuda.py:284] Using Flash Attention backend on V1 engine.
INFO 07-21 12:04:46 [weight_utils.py:292] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  20% Completed | 1/5 [00:00<00:01,  2.15it/s]
Loading safetensors checkpoint shards:  40% Completed | 2/5 [00:00<00:01,  1.99it/s]
Loading safetensors checkpoint shards:  60% Completed | 3/5 [00:01<00:00,  2.74it/s]
Loading safetensors checkpoint shards:  80% Completed | 4/5 [00:01<00:00,  2.67it/s]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:02<00:00,  2.36it/s]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:02<00:00,  2.39it/s]



INFO 07-21 12:04:49 [default_loader.py:272] Loading weights took 2.16 seconds
INFO 07-21 12:04:49 [gpu_model_runner.py:1801] Model loading took 15.2683 GiB and 3.052762 seconds
INFO 07-21 12:04:57 [backends.py:508] Using cache directory: /home/ljy/.cache/vllm/torch_compile_cache/28af56ab07/rank_0_0/backbone for vLLM's torch.compile
INFO 07-21 12:04:57 [backends.py:519] Dynamo bytecode transform time: 7.79 s
INFO 07-21 12:05:03 [backends.py:155] Directly load the compiled graph(s) for shape None from the cache, took 4.980 s
INFO 07-21 12:05:04 [monitor.py:34] torch.compile takes 7.79 s in total
INFO 07-21 12:05:05 [gpu_worker.py:232] Available KV cache memory: 5.66 GiB
INFO 07-21 12:05:05 [kv_cache_utils.py:716] GPU KV cache size: 41,216 tokens
INFO 07-21 12:05:05 [kv_cache_utils.py:720] Maximum concurrency for 40,960 tokens per request: 1.01x


Capturing CUDA graph shapes: 100%|██████████| 67/67 [00:17<00:00,  3.91it/s]


INFO 07-21 12:05:22 [gpu_model_runner.py:2326] Graph capturing finished in 17 secs, took 0.61 GiB
INFO 07-21 12:05:22 [core.py:172] init engine (profile, create kv cache, warmup model) took 33.08 seconds
CoT結果の判定を開始...


Adding requests: 100%|██████████| 4/4 [00:00<00:00, 81.78it/s]





Adding requests: 100%|██████████| 4/4 [00:00<00:00, 71.12it/s]






Adding requests: 100%|██████████| 4/4 [00:00<00:00, 52.51it/s]






Adding requests: 100%|██████████| 4/4 [00:00<00:00, 51.61it/s]




100%|██████████| 4/4 [00:14<00:00,  3.60s/it]

GenSelect結果の判定を開始...


FileNotFoundError: [Errno 2] No such file or directory: './results/inference/genselect'